# Aurora Policing Project

## About the policing data

Throughout this project, I will be analyzing a dataset of traffic stops in Aurora Colorado that was collected by the "Stanford Open Policing Project". [Stanford Open Policing Project Data](https://openpolicing.stanford.edu/data/)



<table>
  <tr>
    <td>Column name</td>
    <td>Column meaning</td>
    <td>Example value</td>
  </tr>
  <tr>
    <td>raw_row_number</td>
    <td>An number used to join clean data back to the raw data</td>
    <td>38299</td>
  </tr>
  <tr>
    <td>date</td>
    <td>The date of the stop, in YYYY-MM-DD format. Some states do not provide
    the exact stop date: for example, they only provide the year or quarter in
    which the stop occurred. For these states, stop_date is set to the date at
    the beginning of the period: for example, January 1 if only year is
    provided.</td>
    <td>"2017-02-02"</td>
  </tr>
  <tr>
    <td>time</td>
    <td>The 24-hour time of the stop, in HH:MM format.</td>
    <td>20:15</td>
  </tr>
  <tr>
    <td>location</td>
    <td>The freeform text of the location. Occasionally, this represents the
    concatenation of several raw fields, i.e. street_number, street_name</td>
    <td>"248 Stockton Rd."</td>
  </tr>
  <tr>
    <td>lat</td>
    <td>The latitude of the stop. If not provided by the department, we
    attempt to geocode any provided address or location using
    Google Maps. Google Maps returns a "best effort" response, which may not
    be completely accurate if the provided location was malformed or
    underspecified. To protect against suprious responses, geocodes more than
    4 standard deviations from the median stop lat/lng are set to NA.
    <td>72.23545</td>
  </tr>
  <tr>
    <td>lng</td>
    <td>The longitude of the stop. If not provided by the department, we
    attempt to geocode any provided address or location using
    Google Maps. Google Maps returns a "best effort" response, which may not
    be completely accurate if the provided location was malformed or
    underspecified. To protect against suprious responses, geocodes more than
    4 standard deviations from the median stop lat/lng are set to NA.
    </td>
    <td>115.2808</td>
  </tr>
  
  <tr>
    <td>district</td>
    <td>Police district. If not provided, but we have retrieved police
    department shapfiles and the location of the stop, we geocode the stop and
    find the district using the shapefiles.</td>
    <td>8</td>
  </tr>
  
  <tr>
    <td>subject_age</td>
    <td>The age of the stopped subject. When date of birth is given, we
    calculate the age based on the stop date. Values outside the range of
    10-110 are coerced to NA.</td>
    <td>54.23</td>
  </tr>
  <tr>
    <td>subject_race</td>
    <td>The race of the stopped subject. Values are standardized to white,
    black, hispanic, asian/pacific islander, and other/unknown</td>
    <td>"hispanic"</td>
  </tr>
  <tr>
    <td>subject_sex</td>
    <td>The recorded sex of the stopped subject.</td>
    <td>"female"</td>
  </tr>
 
  <tr>
    <td>type</td>
    <td>Type of stop: vehicular or pedestrian.</td>
    <td>"vehicular"</td>
  </tr>
  
  <tr>
    <td>violation</td>
    <td>Specific violation of stop where provided. What is recorded here varies
    widely across police departments.</td>
    <td>"SPEEDING 15-20 OVER"</td>
  </tr>
  <tr>
    <td>citation_issued</td>
    <td>Indicates whether a citation was issued.</td>
    <td>TRUE</td>
  </tr>
  
  <tr>
    <td>outcome</td>
    <td>The strictest action taken among arrest, citation, warning, and
    summons.</td>
    <td>"citation"</td>
  </tr>
  
</table>



## Preparing the Aurora policing data for analysis

In [ ]:
# Import numpy library
import numpy as np

In [ ]:
# Import pandas library 
import pandas as pd

In [ ]:
# Import matplotlib.pyplot library 
import matplotlib.pyplot as plt

In [ ]:
# Import seaborn library
import seaborn as sns

In [ ]:
from datetime import datetime

In [ ]:
# Read file into dataframe named data 
data = pd.read_csv("co_aurora_2019_02_25 copy.csv")

### Examing the dataset

In [ ]:
# Examine the head of dataframe 
data.head()

In [ ]:
data.dtypes

#### Aurora District information 
[Aurora District (1,2,3) Map](https://wiki.radioreference.com/images/3/3b/Aurora_Beat_Map.pdf)

## Dropping columns
Dropping the columns which are not useful to analysis. 

In [ ]:
# Count the number of missing values in each column
print(data.isnull().sum())

In [ ]:
data.shape

#### Dropping raw_row_number, because I will not use this number as reference number

In [ ]:
data.drop(['raw_row_number'], axis = 'columns', inplace = True)
print(data.shape)

#### Comparing 'citation_issued' and 'outcome' columns

In [ ]:
data.citation_issued.value_counts()

In [ ]:
data.outcome.value_counts()

In [ ]:
data.shape

All drivers were cited I can drop both 'citation_issued' and 'outcome' columns.

In [ ]:
data.drop(['citation_issued','outcome'], axis = 1 , inplace = True)

#### Dropping lat, lng, and district columns
I will analyze the data for entire Aurora city, so I will not use specific location data.

In [ ]:
# Drop 'lat', and 'lng' columns
data.drop(['lat','lng', 'district','location'], axis = 'columns', inplace = True)
print(data.shape)

I will drop 'type' column because type pedestrian only has 24 rows out of 163803. 

In [ ]:
data.type.value_counts()

#### Dropping rows having 'type' value 'pedestrian'
'type' column has very small portion of 'pedestrian'. I will drop those rows.

In [ ]:
# Check data shape befor dropping
data.shape

In [ ]:
# Drop rows which has pedestrian data
data.drop(data[data['type'] == 'pedestrian'].index, inplace = True)

In [ ]:
data.shape

In [ ]:
# Drop 'type' column
data.drop(['type'], axis = 'columns', inplace = True)

In [ ]:
# Check data shape after dropping column 'type'
data.shape

#### Dropping rows having missing values
I will drop the rows which contain any missing values if the fraction of missing row is small. (less than 5%)

In [ ]:
# Calculate the percentage of missing data 
print(data.isnull().sum()/data.shape[0])

In [ ]:
data.dropna(subset= ['time', 'subject_age', 'subject_race', 'subject_sex', 'violation'], inplace = True)
print(data.shape)

In [ ]:
# Count the number of missing values in each column (again)
print(data.isnull().sum())

In [ ]:
# Examine the shape of the Dataframe
data.shape

In [ ]:
### Age rounded to integer
data['subject_age'] = data['subject_age'].round().astype(int)

In [ ]:
data['subject_age'].head()

In [ ]:
data.head()

### Examining traffic violations

In [ ]:
data.violation.value_counts()

In [ ]:
data.loc[data['violation'].str.contains("Speed") == True,'violation'].value_counts()

In [ ]:
data.loc[data['violation'].str.contains("Speed") == True,'violation'] = "Speeding"

In [ ]:
data.loc[data['violation'].str.contains("Seat Belt")== True, 'violation'].value_counts()

In [ ]:
data.loc[data['violation'].str.contains("Seat Belt")== True, 'violation'] = "Failed to Obey Seat Belt Requirements"

In [ ]:
data.loc[data['violation'].str.contains("Stop Sign")== True, 'violation'].value_counts()

In [ ]:
data.loc[data['violation'].str.contains("Stop Sign")== True, 'violation'] = "Disregarded Stop Sign"

In [ ]:
data.loc[data['violation'].str.contains("License|Registration")== True, 'violation'].value_counts()

In [ ]:
data.loc[data['violation'].str.contains("License")== True, 'violation'] = "License/Registration Violation"

In [ ]:
data.loc[(data['violation'].str.contains("Signal Light|Red|Green")== True), 'violation'] .value_counts()

In [ ]:
data.loc[data['violation'].str.contains("Signal Light|Red Light|Green")== True, 'violation'] = "Failed to Obey Signal Lights"

In [ ]:
data.violation.value_counts()

In [ ]:
data.violation.value_counts(normalize = True)

Almost half of all violations are for speeding, followed by insurance violations and seat belt violations.

In [ ]:
# Add 'count_violation' column to analyze 'violation' column more easily by types of violation.
data['count_violation(%)'] = data['violation'].map(data['violation'].value_counts(normalize = True))

In [ ]:
# Change 'violation' type to 'Others' where the 'count_violation'(% of total violations) is less than 1%

In [ ]:
data.loc[data['count_violation(%)'] < 0.01, 'violation'] = 'Others'

In [ ]:
data['violation'].value_counts()

In [ ]:
data.head()

## About the aurora weather data
[NOAA](https://www.ncdc.noaa.gov/cdo-web/)

<p style='text-align: justify;'> 
I added new weather data to determind if whether conditions have impact on police behavior. The weather data was collected by the National Centers for Environmental Information , and the duration of the weather data is from 01/01/2012 to 12/31/2016.
One possible hypothesis is that weather conditions impact on police behavior during traffic stop. Even though dataset has specific stop locatioin, I would use the data from one station("Centennial International Airpot Station") near Aurora. This is not ideal, but Aurora is a small city and airport weather statation tends to have more weather dataset than the others, it will give me a general idea of the weather condition thourghout the city. </p> 

## Preparing Aurora weather data for analysis

### Examing the dataset

In [ ]:
weather = pd.read_csv('weather.csv')

In [ ]:
weather.head()

* PRCP :  Precipitation
* AWND :  Wind speed
* SNOW, SNWD :  Snowfall, Snow depth
* TAVG, TAX, TMIN :  Temperature
* WT01,WT02, ..., WT09 :  Bad weather conditions 
  
     * 01 = Fog, ice fog, or freezing fog 
     * 02 = Heavy fog or heaving freezing fog 
     * 03 = Thunder
     * 04 = Ice pellets, sleet, snow pellets, or small hail
     * 05 = Hail 
     * 06 = Glaze or rime
     * 08 = Smoke or haze
     * 09 = Blowing or drifting snow
 

In [ ]:
weather.shape

In [ ]:
weather.isnull().sum()

#### Dropping columns 'STATION' , 'NAME', 'SNOW', 'SNOWD'

In [ ]:
weather.STATION.value_counts()

In [ ]:
weather.NAME.value_counts()

Since columns 'STATION' and 'NAME' have only one value 'USW00093067' and 'CENTENNIAL AIRPORT' each, I will drop those two columns. Columns 'SNOW' and 'SNOWD' only have null values, I will also drop those columns.

In [ ]:
# Drop 'STATION' and 'NAME' columns
weather.drop(['STATION','NAME'], axis = 'columns', inplace = True)

In [ ]:
weather.shape

In [ ]:
weather.drop(['SNOW','SNWD'], axis = 'columns', inplace = True )

In [ ]:
weather.shape

<p style =  'text-align: justify; '>
Columns 'TMAX' and 'TMIN'  have to be filled in because everyday has to have max and min temperature. I will caculate the min and max temperature by using 7 consecutive days excluding the day which has null value. </p>

In [ ]:
weather[weather['TMIN'].isna()]

In [ ]:
weather[weather['TMAX'].isna()]

Since null values of 'TMAX' and 'TMIN' are in the same row, I will use the same 7 consecutive days to calculate min and max temperatuer on 2015-10-02. (2015-09-29 ~ 2015-10-05) 

In [ ]:
weather.iloc[1366:1373, :]

In [ ]:
max_temp_for_calculation = weather.iloc[1366:1373,4]

In [ ]:
max_temp_for_calculation

In [ ]:
weather.iloc[1369,4] = max_temp_for_calculation.mean(skipna = True)

In [ ]:
min_temp_for_calculation = weather.iloc[1366:1373,5]

In [ ]:
min_temp_for_calculation

In [ ]:
weather.iloc[1369,5] = min_temp_for_calculation.mean(skipna = True)

In [ ]:
weather.iloc[1366:1373,:]

'TMAX' and 'TMIN' on 2015-10-12 are filled in now.

#### Filliing in average temperature 'TAVG'
I will calculate the values of average temperature ('TAVG') column by calculating the mean of min and max temperature.

In [ ]:
weather['TAVG'] = weather[['TMAX','TMIN']].mean(axis=1)

In [ ]:
weather.head()

In [ ]:
weather.isnull().sum()

### Examing wind speed
Explore the wind data to check for reliability.

In [ ]:
weather['AWND'].head()

In [ ]:
print(weather['AWND'].describe())

Minimum value is above zero, and average value is greter than min and less than max. Also, the numbers are reasonable miles per hour value.

#### Creating a box plot

In [ ]:
weather['AWND'].plot(kind='box')
plt.show()

In [ ]:
weather['AWND'].plot(kind = 'hist', bins=30)
plt.show()

### Examing temperature
Explore the temperature data to check for reliability. Temperature is measured in degrees Fahrenheit.

In [ ]:
print(weather[['TAVG','TMAX','TMIN']].head())

In [ ]:
# Create data frame 'weather_temp' only contains temperature data.
weather_temp = weather[['TAVG','TMAX', 'TMIN']]

In [ ]:
weather_temp.head()

In [ ]:
weather_temp.shape

In [ ]:
print(weather[['TAVG','TMAX','TMIN']].describe())

In [ ]:
weather[['TAVG', 'TMAX', 'TMIN']].plot(kind = 'box')
plt.show()

### Counting bad weather conditions
The weather dataframe contains 8 columns that start with 'WT'. Each of those represents a bad weather condition. For example, "WT05" indicates "Hail" and "WT09" indicates "Blowing or Drifting Snow". For every row in the dataset, each WT column contains either 1 or NaN meaning the weather condition is not obsereved. I will quantify "how bad" the weather was each day by summing the number of 1s in each row.

In [ ]:
# New dataframe 'WT' contains only weather conditions
WT = weather.loc[:,'WT01':'WT09']

In [ ]:
WT.isnull().count()

In [ ]:
# Replace the missing values with 0.
WT = WT.fillna(0).astype('int')

In [ ]:
# Calculate the total number of 1s in each row.
weather['bad_weather'] = WT.sum(axis = 1)

In [ ]:
weather['bad_weather'].head(10)

In [ ]:
# Create a histogram to visulize
weather.bad_weather.plot(kind = 'hist')
plt.show()

It looks like many days did not have any bad weather conditions and only a small portion of days had more than two bad weather conditions.

### Rating the weather conditions
I used the counts to create rating system for bad weather. I counted the unique values in the bad_weather column and sort the index and create a dictioinary called 'mapping' that maps the bad_weather integers to strings. Then I converted the bad_weather integers to strings by using the 'mapping' dictionary and stored the results in the new column called 'rating'.

* Convert 0 to 'Good'
* Convert 1 through 2 to 'Bad'
* Convert 3 through 4 to 'Worse'

In [ ]:
weather.bad_weather.value_counts().sort_index()

In [ ]:
# Create a dictionary that maps integers to string
mapping = {0:'Good', 1:'Bad', 2:'Bad', 3: 'Worse', 4: 'Worse'}

In [ ]:
# Convert the 'bad_weather' integers to string using 'mapping'
weather['weather_rating'] = weather.bad_weather.map(mapping).astype('str')

In [ ]:
weather['weather_rating'].head()

In [ ]:
weather['weather_rating'].value_counts()

#### Changing the data type to category
There is a few poosible values, so for efficiency, I changed the data type to category and gave it a logical order.

In [ ]:
cats = ['Good', 'Bad', 'Worse']

In [ ]:
# Change the data type of 'rating' to category
weather['weather_rating'] = weather.weather_rating.astype(pd.api.types.CategoricalDtype(ordered = True, categories = cats))


In [ ]:
weather.head()

I will use 'rating' column to analyze the effects of weather on police behavior.

### Creating a dataframe 'weather_rating' from 'weather' dataframe for merging
I will take only 'DATE' and 'rating' columns to create 'weather_rating' dataframe and I will merge 'data' and 'weather_rating' dataframes to analyze.

In [ ]:
weather = weather[['DATE','weather_rating']]

In [ ]:
weather.head()

## Merging datasets

In [ ]:
data.head()

In [ ]:
data['date'] = pd.to_datetime(data['date'])

In [ ]:
weather.head()

In [ ]:
weather['DATE'] = pd.to_datetime(weather['DATE'])

In [ ]:
weather['DATE'].head()

In [ ]:
police_weather_data = pd.merge(left = data, right = weather, left_on = 'date', right_on = 'DATE', how = 'left')

In [ ]:
police_weather_data.head()

### Creating a DatetimeIndex
I combined 'date' column and 'time' column to convert datetime format.

In [ ]:
data.date = data.date.astype(str) 

In [ ]:
data.date.head()

In [ ]:
combine = data.date.str.cat(data.time.astype(str), sep = ' ' )

In [ ]:
police_weather_data['datetime'] = pd.to_datetime(combine)

In [ ]:
police_weather_data.set_index('datetime', inplace = True)

In [ ]:
# Examine the index
police_weather_data.head()

In [ ]:
# Dropping unnecessary date or time columns
police_weather_data.drop(['date', 'time', 'Date', 'DATE'], axis = 1, inplace = True)

In [ ]:
police_weather_data.head()

## Does driver gender affect the violation types?
The question I am trying to answer is whether male and female drivers tend to commit different types of traffic violations.

In [ ]:
# Create a dataframe of female driver
female_driver = police_weather_data[police_weather_data.subject_sex == 'female']

In [ ]:
# create a dataframe of male drivers
male_driver = police_weather_data[police_weather_data.subject_sex == 'male']

In [ ]:
# Count the unique violation values of female driver
female_driver.violation.value_counts(normalize = True).head(5)

In [ ]:
# Count unique values of violation of male driver
male_driver.violation.value_counts(normalize = True).head(5)

In Aurora city, women are about 5 percent more likely to get speed tickets than men, other than that all the other categories of women and men seems pretty much similar. 


## Does driver race affect the violation types?
The question I am trying to answer is whether the race types drivers tend to commit different types of traffic violations. I will analyze top 5 violations.

In [ ]:
# Create a dataframe of white drivers
white = police_weather_data[police_weather_data.subject_race == 'white']

In [ ]:
# Create a dataframe of African/American drivers
black = police_weather_data[police_weather_data.subject_race == 'black']

In [ ]:
# Create a dataframe of Asian/Pacific drivers
asian_pacific = police_weather_data[police_weather_data.subject_race == 'asian/pacific islander']

In [ ]:
# Create a dataframe of all the other drivers
other = police_weather_data[police_weather_data.subject_race == 'other/unknown']

In [ ]:
# Compute the violations by white drivers as portioins
white.violation.value_counts(normalize=True).head(8)

In [ ]:
# Compute the violations by African American drivers as portion
black.violation.value_counts(normalize = True).head(8)

In [ ]:
# Compute the violations by Asian drivers as portion
asian_pacific.violation.value_counts(normalize=True).head(8)

In [ ]:
# Compute the violations by all the other race drivers
other.violation.value_counts(normalize=True).head(8)

White driver are more likely to speed than the other race, but all drivers' main resson to get citation is "Speeding".

## Does driver age affect the violation types?
The question I am trying to answer is whether age of driver tend to commit different types of traffic violations.

## Does time of day affect citation rate?

## Does weather affect the citation rate?